## Imports

In [81]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import precision_score, recall_score, f1_score

## Loading Data

In [82]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
background = np.load("background.npz")
background_wash = pd.DataFrame()
background_lou = pd.DataFrame()
def normalize(data):
    stds = np.std(data, axis=-1, keepdims=True)
    return data / stds
for key in background:
    background_wash = background[key][:, 0, :]
    background_wash = torch.tensor(normalize(background_wash), dtype=torch.float32)
    background_lou = background[key][:, 1, :]
    background_lou = torch.tensor(normalize(background_lou), dtype=torch.float32)

bbh_for_challenge = np.load("bbh_for_challenge.npy")
bbh_wash = bbh_for_challenge[:, 0, :]
bbh_wash = torch.tensor(normalize(bbh_wash), dtype=torch.float32)
bbh_lou = bbh_for_challenge[:, 1, :]
bbh_lou = torch.tensor(normalize(bbh_lou), dtype=torch.float32)

sglf_for_challenge = np.load("sglf_for_challenge.npy")
sglf_wash = sglf_for_challenge[:, 0, :]
sglf_wash = torch.tensor(normalize(sglf_wash), dtype=torch.float32)
sglf_lou = sglf_for_challenge[:, 1, :]
sglf_lou = torch.tensor(normalize(sglf_lou), dtype=torch.float32)

https://arxiv.org/pdf/2106.02770 

For my current lab project i'm utilizing this neural process model used in spatiotemporal ML/climate visualizations, which also has a (latent) encoder and a decoder. 

## AutoEncoder

In [83]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 4), 
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(4, input_dim),  
            nn.Sigmoid() 
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

## Energy-Based Model

In [84]:
class EnergyModel(nn.Module):
    def __init__(self, input_dim):
        super(EnergyModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, x):
        return self.network(x)

## Running the Models

### Defined Threshold

In [78]:
train_wash, test_wash = random_split(background_wash, [80000, 20000])
train_lou, test_lou = random_split(background_wash, [80000, 20000])
batch_size = 128
train_loader_wash = DataLoader(train_wash, batch_size=batch_size, shuffle=True)
test_loader_wash = DataLoader(test_wash, batch_size=batch_size, shuffle=False)

train_loader_lou = DataLoader(train_lou, batch_size=batch_size, shuffle=True)
test_loader_lou = DataLoader(test_lou, batch_size=batch_size, shuffle=False)

def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            inputs = batch[0]
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

input_dim = background_wash.shape[1]
autoencoder_wash = Autoencoder(input_dim)
criterion_ae = nn.MSELoss()
optimizer_ae_wash = optim.Adam(autoencoder_wash.parameters(), lr=0.001)

print("\nTraining Autoencoder for Wash:")
train_model(autoencoder_wash, train_loader_wash, criterion_ae, optimizer_ae_wash, epochs=20)

autoencoder_lou = Autoencoder(input_dim)
optimizer_ae_lou = optim.Adam(autoencoder_lou.parameters(), lr=0.001)

print("\nTraining Autoencoder for Lou:")
train_model(autoencoder_lou, train_loader_lou, criterion_ae, optimizer_ae_lou, epochs=20)

energy_model_wash = EnergyModel(input_dim)
criterion_energy = nn.MSELoss()
optimizer_energy_wash = optim.Adam(energy_model_wash.parameters(), lr=0.001)

print("\nTraining Energy-Based Model for Wash:")
train_model(energy_model_wash, train_loader_wash, criterion_energy, optimizer_energy_wash, epochs=20)

# 4. Energy-Based Model for lou
energy_model_lou = EnergyModel(input_dim)
optimizer_energy_lou = optim.Adam(energy_model_lou.parameters(), lr=0.001)

print("\nTraining Energy-Based Model for Lou:")
train_model(energy_model_lou, train_loader_lou, criterion_energy, optimizer_energy_lou, epochs=20)

# def evaluate_model(model, test_loader):
#     model.eval()
#     results = []
#     with torch.no_grad():
#         # If test_loader is a tensor, convert it to a DataLoader
#         if isinstance(test_loader, torch.Tensor):
#             test_loader = DataLoader(test_loader, batch_size=128, shuffle=False)
        
#         for batch in test_loader:
#             # Ensure inputs have the right shape
#             inputs = batch if isinstance(batch, torch.Tensor) else batch[0]
#             if isinstance(model, Autoencoder):
#                 reconstructed = model(inputs)
#                 error = torch.mean((reconstructed - inputs) ** 2, dim=1)
#                 results.extend(error.numpy())
#             else:
#                 energy = model(inputs).squeeze()
#                 results.extend(energy.numpy())
#     return results

def evaluate_model(model, test_loader, percentile_threshold=99):
    model.eval()
    predictions = []
    with torch.no_grad():
        # If test_loader is a tensor, convert it to a DataLoader
        if isinstance(test_loader, torch.Tensor):
            test_loader = DataLoader(test_loader, batch_size=128, shuffle=False)
        
        for batch in test_loader:
            inputs = batch if isinstance(batch, torch.Tensor) else batch[0]
            if isinstance(model, Autoencoder):
                reconstructed = model(inputs)
                error = torch.mean((reconstructed - inputs) ** 2, dim=1)
                predictions.extend(error.numpy())
            else:
                energy = model(inputs).squeeze()
                predictions.extend(energy.numpy())
    
    predictions = np.array(predictions)
    threshold = np.percentile(predictions, percentile_threshold)
    anomalies = predictions > threshold
    print(f"Detected {np.sum(anomalies)} anomalies out of {len(predictions)} samples.")
    return predictions, threshold

datasets = [
    ("Background Wash", test_wash, autoencoder_wash, energy_model_wash),
    ("Background Lou", test_lou, autoencoder_lou, energy_model_lou),
    ("BBH Wash", bbh_wash, autoencoder_wash, energy_model_wash),
    ("BBH Lou", bbh_lou, autoencoder_lou, energy_model_lou),
    ("SGLF Wash", sglf_wash, autoencoder_wash, energy_model_wash),
    ("SGLF Lou", sglf_lou, autoencoder_lou, energy_model_lou),
]

# for name, data, autoencoder, energy_model in datasets:
#     print(f"\nEvaluating {name} with Autoencoder:")
#     reconstruction_errors = evaluate_model(autoencoder, data)
#     print(f"Reconstruction Error - Mean: {np.mean(reconstruction_errors):.4f}, Std: {np.std(reconstruction_errors):.4f}")

#     print(f"Evaluating {name} with Energy-Based Model:")
#     energy_scores = evaluate_model(energy_model, data)
#     print(f"Energy Scores - Mean: {np.mean(energy_scores):.4f}, Std: {np.std(energy_scores):.4f}")

for name, data, autoencoder, energy_model in datasets:
    print(f"\nEvaluating {name} with Autoencoder:")
    recon_errors, threshold_ae = evaluate_model(autoencoder, data)
    print(f"Autoencoder - Threshold: {threshold_ae:.4f}")

    print(f"Evaluating {name} with Energy-Based Model:")
    energy_scores, threshold_energy = evaluate_model(energy_model, data)
    print(f"Energy Model - Threshold: {threshold_energy:.4f}")


Training Autoencoder for Wash:
Epoch 1/20, Loss: 1.1931
Epoch 2/20, Loss: 1.0721
Epoch 3/20, Loss: 1.0178
Epoch 4/20, Loss: 1.0020
Epoch 5/20, Loss: 0.9965
Epoch 6/20, Loss: 0.9941
Epoch 7/20, Loss: 0.9938
Epoch 8/20, Loss: 0.9922
Epoch 9/20, Loss: 0.9921
Epoch 10/20, Loss: 0.9915
Epoch 11/20, Loss: 0.9909
Epoch 12/20, Loss: 0.9904
Epoch 13/20, Loss: 0.9899
Epoch 14/20, Loss: 0.9901
Epoch 15/20, Loss: 0.9900
Epoch 16/20, Loss: 0.9896
Epoch 17/20, Loss: 0.9884
Epoch 18/20, Loss: 0.9883
Epoch 19/20, Loss: 0.9895
Epoch 20/20, Loss: 0.9881

Training Autoencoder for Lou:
Epoch 1/20, Loss: 1.2053
Epoch 2/20, Loss: 1.0812
Epoch 3/20, Loss: 1.0193
Epoch 4/20, Loss: 1.0030
Epoch 5/20, Loss: 0.9966
Epoch 6/20, Loss: 0.9949
Epoch 7/20, Loss: 0.9934
Epoch 8/20, Loss: 0.9921
Epoch 9/20, Loss: 0.9919
Epoch 10/20, Loss: 0.9905
Epoch 11/20, Loss: 0.9906
Epoch 12/20, Loss: 0.9905
Epoch 13/20, Loss: 0.9898
Epoch 14/20, Loss: 0.9893
Epoch 15/20, Loss: 0.9894
Epoch 16/20, Loss: 0.9893
Epoch 17/20, Loss: 

c:\Users\hapah\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/20, Loss: 1.0020
Epoch 2/20, Loss: 1.0005
Epoch 3/20, Loss: 1.0004
Epoch 4/20, Loss: 1.0003
Epoch 5/20, Loss: 1.0002
Epoch 6/20, Loss: 1.0001
Epoch 7/20, Loss: 1.0001
Epoch 8/20, Loss: 1.0001
Epoch 9/20, Loss: 1.0001
Epoch 10/20, Loss: 1.0001
Epoch 11/20, Loss: 1.0001
Epoch 12/20, Loss: 1.0001
Epoch 13/20, Loss: 1.0001
Epoch 14/20, Loss: 1.0001
Epoch 15/20, Loss: 1.0001
Epoch 16/20, Loss: 1.0000
Epoch 17/20, Loss: 1.0000
Epoch 18/20, Loss: 1.0000
Epoch 19/20, Loss: 1.0000
Epoch 20/20, Loss: 1.0000

Training Energy-Based Model for Lou:
Epoch 1/20, Loss: 1.0063
Epoch 2/20, Loss: 1.0006
Epoch 3/20, Loss: 1.0004
Epoch 4/20, Loss: 1.0003
Epoch 5/20, Loss: 1.0002
Epoch 6/20, Loss: 1.0002
Epoch 7/20, Loss: 1.0002
Epoch 8/20, Loss: 1.0001
Epoch 9/20, Loss: 1.0001
Epoch 10/20, Loss: 1.0001
Epoch 11/20, Loss: 1.0001
Epoch 12/20, Loss: 1.0001
Epoch 13/20, Loss: 1.0001
Epoch 14/20, Loss: 1.0001
Epoch 15/20, Loss: 1.0001
Epoch 16/20, Loss: 1.0000
Epoch 17/20, Loss: 1.0000
Epoch 18/20, Loss:

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

### Scuffed Accuracy Model

In [ ]:
train_wash, test_wash = random_split(background_wash, [80000, 20000])
train_bbh_wash, test_bbh_wash = random_split(bbh_wash, [80000, 20000])
train_lou, test_lou = random_split(background_lou, [80000, 20000])
train_bbh_lou, test_bbh_lou = random_split(bbh_lou, [80000, 20000])
batch_size = 64

# DataLoader for Wash
train_loader_wash = DataLoader(train_wash, batch_size=batch_size, shuffle=True)
test_loader_wash = DataLoader(test_wash, batch_size=batch_size, shuffle=False)

train_loader_bbh_wash = DataLoader(train_bbh_wash, batch_size=batch_size, shuffle=True)
test_loader_bbh_wash = DataLoader(test_bbh_wash, batch_size=batch_size, shuffle=False)

# DataLoader for Lou
train_loader_lou = DataLoader(train_lou, batch_size=batch_size, shuffle=True)
test_loader_lou = DataLoader(test_lou, batch_size=batch_size, shuffle=False)

train_loader_bbh_lou = DataLoader(train_bbh_lou, batch_size=batch_size, shuffle=True)
test_loader_bbh_lou = DataLoader(test_bbh_lou, batch_size=batch_size, shuffle=False)

# 2. Training the Autoencoder (Separate Autoencoders for Wash and Lou)
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            inputs = batch[0]
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Autoencoder for Wash and Lou
input_dim = background_wash.shape[1]
autoencoder_wash = Autoencoder(input_dim)
autoencoder_lou = Autoencoder(input_dim)
criterion_ae = nn.MSELoss()
optimizer_ae_wash = optim.Adam(autoencoder_wash.parameters(), lr=0.001)
optimizer_ae_lou = optim.Adam(autoencoder_lou.parameters(), lr=0.001)

# Train Autoencoder for Wash
print("\nTraining Autoencoder for Wash:")
train_model(autoencoder_wash, train_loader_wash, criterion_ae, optimizer_ae_wash, epochs=20)

# Train Autoencoder for Lou
print("\nTraining Autoencoder for Lou:")
train_model(autoencoder_lou, train_loader_lou, criterion_ae, optimizer_ae_lou, epochs=20)

def evaluate_model(model, test_loader, percentile_threshold=95):
    model.eval()
    predictions = []
    with torch.no_grad():
        # If test_loader is a tensor, convert it to a DataLoader
        if isinstance(test_loader, torch.Tensor):
            test_loader = DataLoader(test_loader, batch_size=64, shuffle=False)
        
        for batch in test_loader:
            inputs = batch if isinstance(batch, torch.Tensor) else batch[0]
            if isinstance(model, Autoencoder):
                reconstructed = model(inputs)
                error = torch.mean((reconstructed - inputs) ** 2, dim=1)
                predictions.extend(error.numpy())
            else:
                energy = model(inputs).squeeze()
                predictions.extend(energy.numpy())
    
    predictions = np.array(predictions)
    threshold = np.percentile(predictions, percentile_threshold)
    anomalies = predictions > threshold
    print(f"Detected {np.sum(anomalies)} anomalies out of {len(predictions)} samples.")
    return predictions, threshold

# 3. Prepare Anomaly Detection Data (Separate for Wash and Lou)
def prepare_anomaly_data(autoencoder, normal_data, positive_data):
    autoencoder.eval()
    # Compute reconstruction errors
    normal_errors = evaluate_model(autoencoder, normal_data)
    positive_errors = evaluate_model(autoencoder, positive_data)
    
    # Combine errors and labels
    X = np.concatenate([normal_errors[0], positive_errors[0]])
    y = np.concatenate([np.zeros(len(normal_errors[0])), np.ones(len(positive_errors[0]))])
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)
    # # Shuffle the data
    # indices = np.arange(len(y))
    # np.random.shuffle(indices)
    # X, y = X[indices], y[indices]
    
    # return torch.tensor(X,  dtype=torch.float32), torch.tensor(y,  dtype=torch.float32)

# 4. Train Classifier for Wash
# normal_data_wash = torch.cat([background_wash, bbh_wash])  # Combining background and BBH for normal data
normal_data_wash = background_wash
positive_data_wash = sglf_wash  # Sine-Gaussian for positive anomalies

X_train_wash, y_train_wash = prepare_anomaly_data(autoencoder_wash, normal_data_wash, positive_data_wash)


# Classifier for Wash
classifier_wash = nn.Sequential(
    nn.Linear(1, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

classifier_wash = classifier_wash
# Loss and optimizer for classifier (Wash)
criterion_classifier_wash = nn.BCELoss()
optimizer_classifier_wash = optim.Adam(classifier_wash.parameters(), lr=0.001)

# Train classifier for Wash
for epoch in range(20):
    classifier_wash.train()
    optimizer_classifier_wash.zero_grad()
    outputs = classifier_wash(X_train_wash.view(-1, 1))
    print(X_train_wash)
    print(y_train_wash)
    loss = criterion_classifier_wash(outputs.view(-1), y_train_wash.view(-1))
    loss.backward()
    optimizer_classifier_wash.step()
    print(f"Epoch {epoch+1} (Wash), Loss: {loss.item():.4f}")

# 5. Train Classifier for Lou
normal_data_lou = torch.cat([background_lou, bbh_lou])  # Combining background and BBH for normal data
positive_data_lou = sglf_lou  # Sine-Gaussian for positive anomalies

X_train_lou, y_train_lou = prepare_anomaly_data(autoencoder_lou, normal_data_lou, positive_data_lou)

# Classifier for Lou
classifier_lou = nn.Sequential(
    nn.Linear(1, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
)

# Loss and optimizer for classifier (Lou)
criterion_classifier_lou = nn.BCELoss()
optimizer_classifier_lou = optim.Adam(classifier_lou.parameters(), lr=0.001)

# Train classifier for Lou
for epoch in range(20):
    classifier_lou.train()
    optimizer_classifier_lou.zero_grad()
    outputs = classifier_lou(X_train_lou.view(-1, 1))
    print(X_train_lou)
    loss = criterion_classifier_lou(outputs.view(-1), y_train_lou.view(-1))
    loss.backward()
    optimizer_classifier_lou.step()
    print(f"Epoch {epoch+1} (Lou), Loss: {loss.item():.4f}")

# 6. Evaluate the Classifiers on Test Data (Separate for Wash and Lou)
# Evaluate classifier for Wash
# normal_data_test_wash = torch.cat([background_wash, bbh_wash])  # Combining background and BBH for normal data
normal_data_test_wash = background_wash  # Combining background and BBH for normal data
positive_data_test_wash = sglf_wash  # Sine-Gaussian for positive anomalies

X_test_wash, y_test_wash = prepare_anomaly_data(autoencoder_wash, normal_data_test_wash, positive_data_test_wash)


classifier_wash.eval()
with torch.no_grad():
    predictions_wash = classifier_wash(X_test_wash.view(-1, 1))
    predicted_labels_wash = (predictions_wash > 0.5).float()
    y_test_wash = y_test_wash
    predicted_labels_wash = predicted_labels_wash[:10000]
    y_test_wash = y_test_wash[:10000]
    print(predicted_labels_wash.shape)
    print(y_test_wash.shape)
    accuracy_wash = (predicted_labels_wash == y_test_wash).float().mean()
    precision_wash = precision_score(y_test_wash, predicted_labels_wash)
    recall_wash = recall_score(y_test_wash, predicted_labels_wash)
    f1_wash = f1_score(y_test_wash, predicted_labels_wash)
    
    print(f"Test Accuracy (Wash): {accuracy_wash.item():.4f}")
    print(f"Precision (Wash): {precision_wash:.4f}")
    print(f"Recall (Wash): {recall_wash:.4f}")
    print(f"F1 Score (Wash): {f1_wash:.4f}")

# Evaluate classifier for Lou
normal_data_test_lou = torch.cat([background_lou, bbh_lou])  # Combining background and BBH for normal data
positive_data_test_lou = sglf_lou  # Sine-Gaussian for positive anomalies

X_test_lou, y_test_lou = prepare_anomaly_data(autoencoder_lou, normal_data_test_lou, positive_data_test_lou)

classifier_lou.eval()
with torch.no_grad():
    predictions_lou = classifier_lou(X_test_lou.view(-1, 1))
    predicted_labels_lou = (predictions_lou > 0.5).float()
    y_test_lou = y_test_lou
    predicted_labels_lou = predicted_labels_lou[:10000]
    y_test_lou = y_test_lou[:10000]
    accuracy_lou = (predicted_labels_lou == y_test_lou).float().mean()
    precision_lou = precision_score(y_test_lou, predicted_labels_lou)
    recall_lou = recall_score(y_test_lou, predicted_labels_lou)
    f1_lou = f1_score(y_test_lou, predicted_labels_lou)
    
    print(f"Test Accuracy (Lou): {accuracy_lou.item():.4f}")
    print(f"Precision (Lou): {precision_lou:.4f}")
    print(f"Recall (Lou): {recall_lou:.4f}")
    print(f"F1 Score (Lou): {f1_lou:.4f}")


Training Autoencoder for Wash:
Epoch 1/20, Loss: 1.1273
Epoch 2/20, Loss: 1.0097
Epoch 3/20, Loss: 0.9958
Epoch 4/20, Loss: 0.9927
Epoch 5/20, Loss: 0.9914
Epoch 6/20, Loss: 0.9904
Epoch 7/20, Loss: 0.9898
Epoch 8/20, Loss: 0.9898
Epoch 9/20, Loss: 0.9891
Epoch 10/20, Loss: 0.9888
Epoch 11/20, Loss: 0.9884
Epoch 12/20, Loss: 0.9884
Epoch 13/20, Loss: 0.9887
Epoch 14/20, Loss: 0.9875
Epoch 15/20, Loss: 0.9875
Epoch 16/20, Loss: 0.9877
Epoch 17/20, Loss: 0.9869
Epoch 18/20, Loss: 0.9877
Epoch 19/20, Loss: 0.9871
Epoch 20/20, Loss: 0.9867

Training Autoencoder for Lou:
Epoch 1/20, Loss: 1.1264
Epoch 2/20, Loss: 1.0092
Epoch 3/20, Loss: 0.9953
Epoch 4/20, Loss: 0.9928
Epoch 5/20, Loss: 0.9911
Epoch 6/20, Loss: 0.9906
Epoch 7/20, Loss: 0.9897
Epoch 8/20, Loss: 0.9899
Epoch 9/20, Loss: 0.9898
Epoch 10/20, Loss: 0.9887
Epoch 11/20, Loss: 0.9887
Epoch 12/20, Loss: 0.9883
Epoch 13/20, Loss: 0.9881
Epoch 14/20, Loss: 0.9880
Epoch 15/20, Loss: 0.9881
Epoch 16/20, Loss: 0.9872
Epoch 17/20, Loss: 

c:\Users\hapah\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hapah\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\hapah\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


Test Accuracy (Wash): 1.0000
Precision (Wash): 0.0000
Recall (Wash): 0.0000
F1 Score (Wash): 0.0000
Detected 10000 anomalies out of 200000 samples.
Detected 5000 anomalies out of 100000 samples.
Test Accuracy (Lou): 0.0000
Precision (Lou): 0.0000
Recall (Lou): 0.0000
F1 Score (Lou): 0.0000


c:\Users\hapah\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Anomaly Outlier Detection

In [85]:
# Data preparation
train_wash, test_wash = random_split(background_wash, [80000, 20000])
train_lou, test_lou = random_split(background_wash, [80000, 20000])

batch_size = 128
train_loader_wash = DataLoader(train_wash, batch_size=batch_size, shuffle=True)
test_loader_wash = DataLoader(test_wash, batch_size=batch_size, shuffle=False)
train_loader_lou = DataLoader(train_lou, batch_size=batch_size, shuffle=True)
test_loader_lou = DataLoader(test_lou, batch_size=batch_size, shuffle=False)

# Training function
def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch in train_loader:
            inputs = batch[0]
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, inputs)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader):.4f}")

# Model initialization and training
input_dim = background_wash.shape[1]

autoencoder_wash = Autoencoder(input_dim)
criterion_ae = nn.MSELoss()
optimizer_ae_wash = optim.Adam(autoencoder_wash.parameters(), lr=0.001)

print("\nTraining Autoencoder for Wash:")
train_model(autoencoder_wash, train_loader_wash, criterion_ae, optimizer_ae_wash, epochs=20)

autoencoder_lou = Autoencoder(input_dim)
optimizer_ae_lou = optim.Adam(autoencoder_lou.parameters(), lr=0.001)

print("\nTraining Autoencoder for Lou:")
train_model(autoencoder_lou, train_loader_lou, criterion_ae, optimizer_ae_lou, epochs=20)

energy_model_wash = EnergyModel(input_dim)
criterion_energy = nn.MSELoss()
optimizer_energy_wash = optim.Adam(energy_model_wash.parameters(), lr=0.001)

print("\nTraining Energy-Based Model for Wash:")
train_model(energy_model_wash, train_loader_wash, criterion_energy, optimizer_energy_wash, epochs=20)

energy_model_lou = EnergyModel(input_dim)
optimizer_energy_lou = optim.Adam(energy_model_lou.parameters(), lr=0.001)

print("\nTraining Energy-Based Model for Lou:")
train_model(energy_model_lou, train_loader_lou, criterion_energy, optimizer_energy_lou, epochs=20)

# Anomaly detection using statistical approach
def evaluate_model_statistical(model, test_loader, baseline_data=None, sigma_multiplier=3):
    model.eval()
    predictions = []

    if baseline_data is not None:
        baseline_loader = DataLoader(baseline_data, batch_size=128, shuffle=False) if isinstance(baseline_data, torch.Tensor) else baseline_data
        baseline_scores = []
        with torch.no_grad():
            for batch in baseline_loader:
                inputs = batch if isinstance(batch, torch.Tensor) else batch[0]
                if isinstance(model, Autoencoder):
                    reconstructed = model(inputs)
                    error = torch.mean((reconstructed - inputs) ** 2, dim=1)
                    baseline_scores.extend(error.numpy())
                else:
                    energy = model(inputs).squeeze()
                    baseline_scores.extend(energy.numpy())
        
        baseline_scores = np.array(baseline_scores)
        mean_baseline = np.mean(baseline_scores)
        std_baseline = np.std(baseline_scores)
        threshold = mean_baseline + sigma_multiplier * std_baseline
    else:
        threshold = None

    with torch.no_grad():
        if isinstance(test_loader, torch.Tensor):
            test_loader = DataLoader(test_loader, batch_size=128, shuffle=False)
        for batch in test_loader:
            inputs = batch if isinstance(batch, torch.Tensor) else batch[0]
            if isinstance(model, Autoencoder):
                reconstructed = model(inputs)
                error = torch.mean((reconstructed - inputs) ** 2, dim=1)
                predictions.extend(error.numpy())
            else:
                energy = model(inputs).squeeze()
                predictions.extend(energy.numpy())
                
    with torch.no_grad():
        # If test_loader is a tensor, convert it to a DataLoader
        if isinstance(test_loader, torch.Tensor):
            test_loader = DataLoader(test_loader, batch_size=128, shuffle=False)
        
        for batch in test_loader:
            inputs = batch if isinstance(batch, torch.Tensor) else batch[0]
            if isinstance(model, Autoencoder):
                reconstructed = model(inputs)
                error = torch.mean((reconstructed - inputs) ** 2, dim=1)
                predictions.extend(error.numpy())
            else:
                energy = model(inputs).squeeze()
                predictions.extend(energy.numpy())
    
    predictions = np.array(predictions)

    if threshold is not None:
        anomalies = predictions > threshold
        print(f"Detected {np.sum(anomalies)} anomalies out of {len(predictions)} samples.")
    else:
        anomalies = np.zeros_like(predictions, dtype=bool)

    return predictions, threshold, anomalies

datasets = [
    ("Background Wash", background_wash, autoencoder_wash, energy_model_wash),
    ("Background Lou", background_lou, autoencoder_lou, energy_model_lou),
    ("BBH Wash", bbh_wash, autoencoder_wash, energy_model_wash),
    ("BBH Lou", bbh_lou, autoencoder_lou, energy_model_lou),
    ("SGLF Wash", sglf_wash, autoencoder_wash, energy_model_wash),
    ("SGLF Lou", sglf_lou, autoencoder_lou, energy_model_lou),
]

for name, data, autoencoder, energy_model in datasets:
    print(f"\nEvaluating {name} with Autoencoder:")
    recon_errors, threshold_ae, anomalies_ae = evaluate_model_statistical(autoencoder, data, baseline_data=background_wash)
    print(f"Autoencoder - Threshold: {threshold_ae:.4f}, Anomalies: {np.sum(anomalies_ae)}")

    print(f"Evaluating {name} with Energy-Based Model:")
    energy_scores, threshold_energy, anomalies_energy = evaluate_model_statistical(energy_model, data, baseline_data=background_wash)
    print(f"Energy Model - Threshold: {threshold_energy:.4f}, Anomalies: {np.sum(anomalies_energy)}")



Training Autoencoder for Wash:
Epoch 1/20, Loss: 1.1898
Epoch 2/20, Loss: 1.0669
Epoch 3/20, Loss: 1.0174
Epoch 4/20, Loss: 1.0009
Epoch 5/20, Loss: 0.9970
Epoch 6/20, Loss: 0.9947
Epoch 7/20, Loss: 0.9934
Epoch 8/20, Loss: 0.9923
Epoch 9/20, Loss: 0.9921
Epoch 10/20, Loss: 0.9914
Epoch 11/20, Loss: 0.9911
Epoch 12/20, Loss: 0.9906
Epoch 13/20, Loss: 0.9893
Epoch 14/20, Loss: 0.9896
Epoch 15/20, Loss: 0.9896
Epoch 16/20, Loss: 0.9894
Epoch 17/20, Loss: 0.9894
Epoch 18/20, Loss: 0.9888
Epoch 19/20, Loss: 0.9900
Epoch 20/20, Loss: 0.9884

Training Autoencoder for Lou:
Epoch 1/20, Loss: 1.1895
Epoch 2/20, Loss: 1.0724
Epoch 3/20, Loss: 1.0196
Epoch 4/20, Loss: 1.0024
Epoch 5/20, Loss: 0.9972
Epoch 6/20, Loss: 0.9942
Epoch 7/20, Loss: 0.9935
Epoch 8/20, Loss: 0.9925
Epoch 9/20, Loss: 0.9911
Epoch 10/20, Loss: 0.9912
Epoch 11/20, Loss: 0.9901
Epoch 12/20, Loss: 0.9908
Epoch 13/20, Loss: 0.9909
Epoch 14/20, Loss: 0.9891
Epoch 15/20, Loss: 0.9891
Epoch 16/20, Loss: 0.9900
Epoch 17/20, Loss: 

c:\Users\hapah\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/20, Loss: 1.0015
Epoch 2/20, Loss: 1.0006
Epoch 3/20, Loss: 1.0005
Epoch 4/20, Loss: 1.0005
Epoch 5/20, Loss: 1.0004
Epoch 6/20, Loss: 1.0003
Epoch 7/20, Loss: 1.0002
Epoch 8/20, Loss: 1.0001
Epoch 9/20, Loss: 1.0001
Epoch 10/20, Loss: 1.0001
Epoch 11/20, Loss: 1.0001
Epoch 12/20, Loss: 1.0001
Epoch 13/20, Loss: 1.0001
Epoch 14/20, Loss: 1.0001
Epoch 15/20, Loss: 1.0001
Epoch 16/20, Loss: 1.0001
Epoch 17/20, Loss: 1.0001
Epoch 18/20, Loss: 1.0000
Epoch 19/20, Loss: 1.0000
Epoch 20/20, Loss: 1.0000

Training Energy-Based Model for Lou:
Epoch 1/20, Loss: 1.0016
Epoch 2/20, Loss: 1.0005
Epoch 3/20, Loss: 1.0003
Epoch 4/20, Loss: 1.0002
Epoch 5/20, Loss: 1.0002
Epoch 6/20, Loss: 1.0002
Epoch 7/20, Loss: 1.0001
Epoch 8/20, Loss: 1.0001
Epoch 9/20, Loss: 1.0001
Epoch 10/20, Loss: 1.0001
Epoch 11/20, Loss: 1.0001
Epoch 12/20, Loss: 1.0001
Epoch 13/20, Loss: 1.0001
Epoch 14/20, Loss: 1.0001
Epoch 15/20, Loss: 1.0001
Epoch 16/20, Loss: 1.0001
Epoch 17/20, Loss: 1.0001
Epoch 18/20, Loss: